### Objective
I will cleaning and exploring this dataset using pandas. It will be visualised using matplotlib. Afterwards, I'll hypothesise the relationship and trends of this data. This dataset is obtained via [kaggle.com](https://www.kaggle.com/datasets/bharatnatrayn/movies-dataset-for-feature-extracion-prediction/data).


#### Importing Libraries

In [438]:
import pandas as pd
import numpy as np
import re
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

#### Import Raw Dataset

In [439]:
df = pd.read_csv('Data/raw_movies.csv')
display(df.head())

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\r\nAction, Horror, Thriller",6.1,\r\nA woman with a mysterious illness is force...,\r\n Director:\r\nPeter Thorwarth\r\n| \r\n...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\r\nAnimation, Action, Adventure",5.0,\r\nThe war for Eternia begins again in what m...,"\r\n \r\n Stars:\r\nChris Wood, ...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\r\nDrama, Horror, Thriller",8.2,\r\nSheriff Deputy Rick Grimes wakes up from a...,\r\n \r\n Stars:\r\nAndrew Linco...,"885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\r\nAnimation, Adventure, Comedy",9.2,\r\nAn animated series that follows the exploi...,\r\n \r\n Stars:\r\nJustin Roila...,"414,849",23.0,NaN
4,Army of Thieves,(2021),"\r\nAction, Crime, Horror",NaN,"\r\nA prequel, set before the events of Army o...",\r\n Director:\r\nMatthias Schweighöfer\r\n...,NaN,NaN,NaN


### Data Inspection

In [440]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


(9999, 9)

In [441]:
df.describe()

,RATING,RunTime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


In [442]:
df.isnull().sum()

MOVIES         0
YEAR         644
GENRE         80
RATING      1820
ONE-LINE       0
STARS          0
VOTES       1820
RunTime     2958
Gross       9539
dtype: int64

#### Observations
- There are 9 columns, with 9999 rows
- Most columns stores object
- RATING, RunTime: float64
- Columns w/ NULL: YEAR, GENRE, RATING, VOTES, RunTime, Gross

### Renaming Columns

In [443]:
df_col = df.rename(columns=
          {'MOVIES': 'Movies', 'YEAR': 'Year',
           'GENRE': 'Genre', 'RATING': 'Rating',
           'ONE-LINE': 'Short Desc', 'STARS': 'Stars',
           'VOTES': 'Votes', 'RunTime': 'Run Time',
           'Gross': 'Gross'})
print(df_col.columns)

Index(['Movies', 'Year', 'Genre', 'Rating', 'Short Desc', 'Stars', 'Votes',
       'Run Time', 'Gross'],
      dtype='object')


### Removing Duplicates

In [444]:
print(f'Duplicates: {df_col.duplicated().sum()}')
df_dupe = df_col.drop_duplicates()

Duplicates: 431


### Fixing Data Types
Votes: Remove ',' - convert to integer.

Gross: Remove '$' and 'M' - convert to float.

In [445]:
df_dupe.loc[:, 'Votes'] = df_dupe['Votes'].str.replace(',', '').apply(lambda x: int(x) if x is not np.nan else x)
df_dupe.loc[:, 'Gross'] = df_dupe['Gross'].str.replace('$', '').str.replace('M', '')

In [446]:
df_dupe = df_dupe.astype({'Movies': 'object',
                          'Year': 'object',
                          'Genre': 'object',
                          'Rating': 'float64',
                          'Short Desc': 'object',
                          'Stars': 'object',
                          'Votes': 'Int64',
                          'Run Time': 'float64',
                          'Gross': 'float64'})

In [447]:
df_fix = df_dupe.copy()
df_fix.info()

filtered_df_fix = df_fix[df_fix['Gross'].notnull()]
display(df_fix.head(3))

<class 'pandas.core.frame.DataFrame'>
Index: 9568 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9568 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       9490 non-null   object 
 3   Rating      8168 non-null   float64
 4   Short Desc  9568 non-null   object 
 5   Stars       9568 non-null   object 
 6   Votes       8168 non-null   Int64  
 7   Run Time    7008 non-null   float64
 8   Gross       460 non-null    float64
dtypes: Int64(1), float64(3), object(5)
memory usage: 756.8+ KB


,Movies,Year,Genre,Rating,Short Desc,Stars,Votes,Run Time,Gross
0,Blood Red Sky,(2021),"\r\nAction, Horror, Thriller",6.1,\r\nA woman with a mysterious illness is force...,\r\n Director:\r\nPeter Thorwarth\r\n| \r\n...,21062,121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\r\nAnimation, Action, Adventure",5.0,\r\nThe war for Eternia begins again in what m...,"\r\n \r\n Stars:\r\nChris Wood, ...",17870,25.0,NaN
2,The Walking Dead,(2010–2022),"\r\nDrama, Horror, Thriller",8.2,\r\nSheriff Deputy Rick Grimes wakes up from a...,\r\n \r\n Stars:\r\nAndrew Linco...,885805,44.0,NaN


### Handling Missing Values
Gross & Run Time are the least important to the average viewers. Hence, the rows with missing values for any of these columns will not be removed.

Missing Values for Ratings & Votes will be replaced by the mean average for all recorded movies in the dataset.

All rows with missing Year will be removed.

In [448]:
# Drop rows with missing Year
df_mis = df_fix.dropna(subset=['Year'])

# Fill missing Ratings and Votes with mean
df_mis.loc[:, 'Rating'] = df_mis['Rating'].fillna(df_mis['Rating'].mean())
df_mis.loc[:, 'Votes'] = df_mis['Votes'].fillna(round(df_mis['Votes'].mean()))

In [449]:
df_mis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9026 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       8989 non-null   object 
 3   Rating      9026 non-null   float64
 4   Short Desc  9026 non-null   object 
 5   Stars       9026 non-null   object 
 6   Votes       9026 non-null   Int64  
 7   Run Time    6987 non-null   float64
 8   Gross       460 non-null    float64
dtypes: Int64(1), float64(3), object(5)
memory usage: 714.0+ KB


### Formatting Text Data
Stars: Format from strings into dictionary:
- 'Director' & 'Stars' are keys.
- Names are stored in an array.

Example: {'Director': ['Person A', ..., 'Person N'], 'Stars': ['Person C', ... , 'Person M']}

- Genre: Remove \r\n - convert into an array.
- Short Desc: Remove \r\n.
- Rating: Round to 2 decimal place.
- Year: Convert into a tuple (StartYear, EndYear)
- Split Stars column into two: Director & Stars

In [450]:
def clean_genre(text):
    if type(text) == str:
        return text.strip().replace('\r\n', '').split(',')

In [451]:
def clean_short_desc(text):
    return text.strip().replace('\r\n', '')

In [452]:
def clean_rating(val):
    return round(val, 2)

In [453]:
def clean_year(val):
    val = val.replace('(', '').replace(')', '')
    for i in val:
        if not i.isdigit() and i != '–':
            val = val.replace(i, '')
            
    if '–' not in val:
        try:
            return [int(val)]
        except ValueError:
            return [np.nan]
    arr = val.split('–')
    arr = [arr[0], np.nan] if arr[1] == '' or arr[1] == ' ' else [arr[0], arr[1]]
    
    arr = [int(x) if pd.notna(x) else x for x in arr]
    return arr

In [454]:
df_mis.loc[:, 'Year'] = df_mis['Year'].apply(clean_year)

In [455]:
df_mis.loc[:, 'Genre'] = df_mis['Genre'].apply(clean_genre)

In [456]:
df_mis.loc[:, 'Short Desc'] = df_mis['Short Desc'].apply(clean_short_desc)

In [457]:
df_mis.loc[:, 'Rating'] = df_mis['Rating'].apply(clean_rating)

In [458]:
def clean_stars_old(text):
    # Initialize dictionary
    dict = {'Director': [], 'Stars': []}
    
    # Remove whitespaces & labels (Director, Stars)
    removed_whitespace = text.strip().replace('\r\n', '').replace(' ', '')
    removed_labels = removed_whitespace.replace('Director:', '').replace('Stars:', '')
    
    # Split Director and Stars
    split_labels = removed_labels.split('|')
    
    # Split each element into an array
    if len(split_labels) == 2:
        dict.update({'Director': split_labels[0], 'Stars': split_labels[1]})
        for i in dict:
            dict[i] = dict[i].split(',')
    else:
        dict.update({'Director': split_labels[0], 'Stars': None})
    
        
    return dict

In [459]:
def clean_stars(text):
    # Initialize dictionary
    dict = {'Director': [], 'Stars': []}
    
    # Remove whitespaces
    removed_whitespace = text.strip().replace('\r\n', '').replace(' ', '')
    
    # Split Director and Stars
    if '|' in removed_whitespace:
        removed_labels = removed_whitespace.replace('Director:', '').replace('Stars:', '')
        split_labels = removed_labels.split('|')
        
        # Split each element into an array
        if len(split_labels) == 2:
            dict.update({'Director': split_labels[0], 'Stars': split_labels[1]})
            for i in dict:
                dict[i] = dict[i].split(',')
        else:
            dict.update({'Director': split_labels[0], 'Stars': np.nan})
    
    # If there is only Directors    
    elif 'Stars:' not in removed_whitespace:
        removed_labels = removed_whitespace.replace('Director:', '')
        split_labels = removed_labels.split(',')        
        dict.update({'Director': split_labels, 'Stars': np.nan})
    
    # If there is only Stars    
    elif 'Director:' not in removed_whitespace:
        removed_labels = removed_whitespace.replace('Stars:', '')
        split_labels = removed_labels.split(',')
        dict.update({'Director': np.nan, 'Stars': split_labels})
                
    # Format the first and last name           
    for key, val in dict.items():
        if isinstance(val, list):
            dict[key] = [re.sub(r'(?<!^)(?=[A-Z])', ' ', name) if isinstance(name, str) and not pd.isna(name) else name for name in val]
        elif isinstance(val, str) and not pd.isna(val):
            dict[key] = re.sub(r'(?<!^)(?=[A-Z])', ' ', val)
                    
    return dict

In [460]:
def create_director_list(text):
    dict = clean_stars(text)
    return dict['Director']

In [461]:
def create_star_list(text):
    dict = clean_stars(text)
    return dict['Stars']

In [462]:
# {'Director': ['Person A', ..., 'Person N'], 'Stars': ['Person C', ... , 'Person M']}
df_mis.loc[:, 'Directors'] = df_mis['Stars'].apply(create_director_list)
df_mis.loc[:, 'Stars'] = df_mis['Stars'].apply(create_star_list)

C:\Users\frasid\AppData\Local\Temp\ipykernel_9308\2775121614.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [463]:
df_format = df_mis.copy()
df_format = df_format[['Movies', 'Year', 'Genre', 'Rating', 'Short Desc', 'Votes', 'Run Time', 'Gross', 'Directors', 'Stars']]
display(df_format.head())
df_format.info()

,Movies,Year,Genre,Rating,Short Desc,Votes,Run Time,Gross,Directors,Stars
0,Blood Red Sky,[2021],"[Action, Horror, Thriller]",6.10,A woman with a mysterious illness is forced in...,21062,121.0,NaN,[Peter Thorwarth],"[Peri Baumeister, Carl Anton Koch, Alexander S..."
1,Masters of the Universe: Revelation,"[2021, nan]","[Animation, Action, Adventure]",5.00,The war for Eternia begins again in what may b...,17870,25.0,NaN,NaN,"[Chris Wood, Sarah Michelle Gellar, Lena Heade..."
2,The Walking Dead,"[2010, 2022]","[Drama, Horror, Thriller]",8.20,Sheriff Deputy Rick Grimes wakes up from a com...,885805,44.0,NaN,NaN,"[Andrew Lincoln, Norman Reedus, Melissa Mc Bri..."
3,Rick and Morty,"[2013, nan]","[Animation, Adventure, Comedy]",9.20,An animated series that follows the exploits o...,414849,23.0,NaN,NaN,"[Justin Roiland, Chris Parnell, Spencer Gramme..."
4,Army of Thieves,[2021],"[Action, Crime, Horror]",6.92,"A prequel, set before the events of Army of th...",15144,NaN,NaN,[Matthias Schweighöfer],"[Matthias Schweighöfer, Nathalie Emmanuel, Rub..."


<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 9998
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9026 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       8989 non-null   object 
 3   Rating      9026 non-null   float64
 4   Short Desc  9026 non-null   object 
 5   Votes       9026 non-null   Int64  
 6   Run Time    6987 non-null   float64
 7   Gross       460 non-null    float64
 8   Directors   6284 non-null   object 
 9   Stars       8511 non-null   object 
dtypes: Int64(1), float64(3), object(6)
memory usage: 784.5+ KB


### Handling Outliers

In [464]:
df_format.describe()

,Rating,Votes,Run Time,Gross
count,9026.000000,9026.0,6987.000000,460.000000
mean,6.919727,15144.375028,68.942608,43.701870
std,1.160531,66684.186397,47.310346,82.423304
min,1.100000,5.0,1.000000,0.000000
25%,6.300000,201.0,37.000000,0.150000
50%,6.920000,1068.0,60.000000,6.145000
75%,7.700000,7882.25,95.000000,46.947500
max,9.900000,1713028.0,853.000000,504.010000


From above, we can see that 'Votes', 'Run Time', and 'Gross' contain outliers. This is evident from observing the mean and their minimum and maximum values.

- Votes - Mean: 15144, Min: 5
- Run Time - Mean: 69, Max: 853
- Gross - Mean: 44, Max: 504

It is very difficult to determine if some data were the result of human errors as the outliers can be considered reasonable values of their respective attributes. I will be visualising the data and apply statistical methods to identify these outliers.

In [465]:
columns = ['Rating', 'Votes', 'Run Time', 'Gross']

# Create 2x2 subplot grid
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=columns
)

# Loop through the columns and add box plots
for i, col in enumerate(columns):
    row = i // 2 + 1
    col_pos = i % 2 + 1

    # Create box plot using go.Box for control
    fig.add_trace(
        go.Box(y=df_format[col], name=col),
        row=row, col=col_pos
    )

# Update layout
fig.update_layout(
    height=700,
    width=900,
    title_text="Box Plots of Numeric Columns",
    showlegend=False
)

fig.show()

In [466]:
columns = ['Rating', 'Votes', 'Run Time', 'Gross']

# Create 2x2 subplot grid for histograms
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=columns
)

for i, col in enumerate(columns):
    row = i // 2 + 1
    col_pos = i % 2 + 1

    fig.add_trace(
        go.Histogram(x=df_format[col], name=col),
        row=row, col=col_pos
    )

fig.update_layout(
    height=700,
    width=900,
    title_text="Histograms of Numeric Columns",
    showlegend=False
)

fig.show()


In [467]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

columns = ['Votes', 'Run Time', 'Gross', 'Rating']
x_axis = 'Movies'

# Create subplot: 2 rows, 2 columns (we’ll fill 3 plots + 1 empty)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f"{x_axis} vs {col}" for col in columns] + ['']
)

for i, col in enumerate(columns):
    row = i // 2 + 1
    col_pos = i % 2 + 1

    fig.add_trace(
        go.Scatter(
            x=df_format[x_axis],
            y=df_format[col],
            mode='markers',
            name=f"{x_axis} vs {col}"
        ),
        row=row, col=col_pos
    )
fig.update_xaxes(showticklabels=False)
fig.update_layout(
    height=700,
    width=900,
    title_text=f"Scatter Plots: {x_axis} vs Other Columns",
    showlegend=False
)

fig.show()


In [468]:
def find_outliers_IQR(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    outliers = col[(col < q1 - 1.5 * iqr) | (col > q3 + 1.5 * iqr)]
    return outliers

In [469]:
columns = ['Rating', 'Votes', 'Run Time', 'Gross']

for col in columns:
    outliers = find_outliers_IQR(df_format[col])
    print(f'Column: {col}')
    print(f'Num. of outliers: {len(outliers)}')
    print(f'Max. outliers: {outliers.max()}')
    print(f'Min. outliers: {outliers.min()}\n')

Column: Rating
Num. of outliers: 222
Max. outliers: 9.9
Min. outliers: 1.1

Column: Votes
Num. of outliers: 882
Max. outliers: 1713028
Min. outliers: 19411

Column: Run Time
Num. of outliers: 105
Max. outliers: 853.0
Min. outliers: 187.0

Column: Gross
Num. of outliers: 50
Max. outliers: 504.01
Min. outliers: 117.72



#### Outlier Handling Conclusion
Based on the graphs above, we know that the number of votes, their rating, and their gross, are proportional. Hence, the dataset is reasonable. While it is tempting to remove or replace the outliers with another values, we need to acknowledge that this dataset is obtained through a reputable source with minimal human errors (automated). Instead, we will be transforming these data to reduce skewness and variance while maintaining credible data. This will aid in better data visualisation before performing hypothesis.

### Filtering Irrelevant Data
For this dataset, we will be keeping all of the columns (including the new ones) to aid in discovering trends and relationships between the attributes.

### Validating Data Consistency

In [470]:
# Check datatypes of columns
display(df_format)
df_format.info(verbose=True)

,Movies,Year,Genre,Rating,Short Desc,Votes,Run Time,Gross,Directors,Stars
0,Blood Red Sky,[2021],"[Action, Horror, Thriller]",6.10,A woman with a mysterious illness is forced in...,21062,121.0,NaN,[Peter Thorwarth],"[Peri Baumeister, Carl Anton Koch, Alexander S..."
1,Masters of the Universe: Revelation,"[2021, nan]","[Animation, Action, Adventure]",5.00,The war for Eternia begins again in what may b...,17870,25.0,NaN,NaN,"[Chris Wood, Sarah Michelle Gellar, Lena Heade..."
2,The Walking Dead,"[2010, 2022]","[Drama, Horror, Thriller]",8.20,Sheriff Deputy Rick Grimes wakes up from a com...,885805,44.0,NaN,NaN,"[Andrew Lincoln, Norman Reedus, Melissa Mc Bri..."
3,Rick and Morty,"[2013, nan]","[Animation, Adventure, Comedy]",9.20,An animated series that follows the exploits o...,414849,23.0,NaN,NaN,"[Justin Roiland, Chris Parnell, Spencer Gramme..."
4,Army of Thieves,[2021],"[Action, Crime, Horror]",6.92,"A prequel, set before the events of Army of th...",15144,NaN,NaN,[Matthias Schweighöfer],"[Matthias Schweighöfer, Nathalie Emmanuel, Rub..."
...,...,...,...,...,...,...,...,...,...,...
9993,Totenfrau,"[2022, nan]","[Drama, Thriller]",6.92,Add a Plot,15144,NaN,NaN,[Nicolai Rohde],"[Felix Klare, Romina Küper, Anna Maria Mühe, R..."
9995,Arcane,"[2021, nan]","[Animation, Action, Adventure]",6.92,Add a Plot,15144,NaN,NaN,[],NaN
9996,Heart of Invictus,"[2022, nan]","[Documentary, Sport]",6.92,Add a Plot,15144,NaN,NaN,[Orlandovon Einsiedel],[Star: Prince Harry]
9997,The Imperfects,"[2021, nan]","[Adventure, Drama, Fantasy]",6.92,Add a Plot,15144,NaN,NaN,[Jovanka Vuckovic],"[Morgan Taylor Campbell, Iñaki Godoy, Rhianna ..."


<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 9998
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movies      9026 non-null   object 
 1   Year        9026 non-null   object 
 2   Genre       8989 non-null   object 
 3   Rating      9026 non-null   float64
 4   Short Desc  9026 non-null   object 
 5   Votes       9026 non-null   Int64  
 6   Run Time    6987 non-null   float64
 7   Gross       460 non-null    float64
 8   Directors   6284 non-null   object 
 9   Stars       8511 non-null   object 
dtypes: Int64(1), float64(3), object(6)
memory usage: 784.5+ KB


#### Export Clean Dataset to XLSX & CSV

In [471]:
df_format.to_csv('Data/cleaned_movies.csv', index=False)
df_format.to_excel('Data/cleaned_movies.xlsx', index=False)